## Step by step api's methods description

In [2]:
import os
import re
import pandas as pd
import numpy as np
import qiime2
from qiime2 import Artifact
from qiime2.plugins import metadata, feature_table, diversity, emperor
from q2_emperor import plot, procrustes_plot, biplot, generic_plot
from scipy.spatial.distance import squareform, pdist
import skbio

In [3]:
!ls ..

Dockerfile  Makefile  README.md  api  notebooks  run_server.sh


In [4]:
!pwd

/ClusterApp/notebooks


In [5]:
# Change directory to import function from api
os.chdir('..')

In [6]:
os.getcwd()

'/ClusterApp'

In [7]:
os.listdir()

['.git',
 'notebooks',
 'Makefile',
 'README.md',
 'run_server.sh',
 '.ipynb_checkpoints',
 'api',
 'Dockerfile',
 '.gitignore']

In [8]:
from api.utils import qiime2PCoA
from api.gnps import Proteosafe

In [9]:
taskid = 'a0b4377f6e4540268602c30fedf69b0b'
# Can you see the available options in the code? We have to create a 
# drop-down menu for the user to choose
workflow = 'FBMN'
metric = 'euclidean'
gnps_result = Proteosafe(taskid, workflow)
gnps_result.get_gnps()

In [10]:
gnps_result.meta.head()

,filename,ATTRIBUTE_Sample_Number,ATTRIBUTE_Order_Injection_2965_3788,ATTRIBUTE_Plot,ATTRIBUTE_Forest_Region,ATTRIBUTE_Pertubation,ATTRIBUTE_Super_Order_APGIV_Clade,ATTRIBUTE_Order_APGIV_Order,ATTRIBUTE_Family,ATTRIBUTE_Genus,...,ATTRIBUTE_Order_APGIII,ATTRIBUTE_CS,ATTRIBUTE_AB,ATTRIBUTE_LIG,ATTRIBUTE_AFE,ATTRIBUTE_CMSF,ATTRIBUTE_Asat,ATTRIBUTE_C,ATTRIBUTE_N,ATTRIBUTE_C/N
0,10_3uL_30_05_1-33_01_3004.mzXML,10,3004,T,Secondary,Clearcut,Monocots,Arecales,Arecaceae,Euterpe,...,0.105,0.135,0.016,0.387,0.120,456.674,32.262,14.155,NaN,NaN
1,11_3uL_01_06_1-42_01_3131.mzXML,11,3131,T,Secondary,Clearcut,Monocots,Arecales,Arecaceae,Euterpe,...,0.105,0.135,0.016,0.387,0.120,456.674,32.262,14.155,NaN,NaN
2,12_3uL_30_05_1-68_01_3052.mzXML,12,3052,T,Secondary,Clearcut,Monocots,Arecales,Arecaceae,Euterpe,...,0.105,0.135,0.016,0.387,0.120,456.674,32.262,14.155,NaN,NaN
3,78_3uL_03_07_1-66_01_3757.mzXML,78,3757,K,Primary,Preserved,Monocots,Arecales,Arecaceae,Euterpe,...,0.116,0.211,0.010,0.347,0.064,484.183,23.933,20.231,NaN,NaN
4,79_3uL_03_07_1-17_01_3699.mzXML,79,3699,K,Primary,Preserved,Monocots,Arecales,Arecaceae,Euterpe,...,0.116,0.211,0.010,0.347,0.064,484.183,23.933,20.231,NaN,NaN


In [11]:
gnps_result.feat.head()

,row ID,row m/z,row retention time,12_3uL_30_05_1-68_01_3052.mzXML Peak area,78_3uL_03_07_1-66_01_3757.mzXML Peak area,11_3uL_01_06_1-42_01_3131.mzXML Peak area,10_3uL_30_05_1-33_01_3004.mzXML Peak area,80_3uL_30_05_1-58_01_3033.mzXML Peak area,153_3uL_01_06_1-23_01_3111.mzXML Peak area,152_3uL_01_06_1-87_01_3074.mzXML Peak area,79_3uL_03_07_1-17_01_3699.mzXML Peak area,224_3uL_03_07_1-81_01_3774.mzXML Peak area,154_3uL_01_06_1-37_01_3125.mzXML Peak area,225_3uL_03_07_1-92_01_3787.mzXML Peak area,226_3uL_03_07_1-6_01_3686.mzXML Peak area,Unnamed: 15
0,5,353.265401,35.119569,1.099282e+06,1631616.110,1.281668e+06,860049.175,1254956.950,1.010253e+06,900838.985,1.441528e+06,1.391126e+06,1.317142e+06,1.292446e+06,2443.4200,NaN
1,7,381.296298,37.262528,9.998174e+05,1445426.520,1.249140e+06,915588.990,1234167.890,9.921647e+05,800190.520,1.270731e+06,1.384528e+06,1.256917e+06,1.198393e+06,303.0000,NaN
2,8,581.148154,7.884651,1.348523e+06,1536205.639,5.228738e+06,1779064.237,127515.698,1.108239e+05,36034.871,2.423437e+04,1.746068e+06,1.812014e+05,3.515917e+04,338.3965,NaN
3,10,609.268397,35.247439,5.505347e+05,1259964.695,1.099087e+06,1900706.545,525107.070,4.631662e+05,325802.580,3.167505e+05,3.375690e+06,6.051916e+05,1.972567e+06,0.0000,NaN
4,13,305.156498,49.782424,6.643086e+05,299720.325,8.127709e+05,559341.440,714114.810,9.085950e+05,683273.360,4.172964e+05,3.073419e+05,7.009724e+05,4.861288e+05,0.0000,NaN


In [12]:
if not os.path.exists(f'notebooks/{taskid}'):
    os.mkdir(f'notebooks/{taskid}')

pcoa_obj = qiime2PCoA(gnps_result.meta, gnps_result.feat,
                      out_dir=f'notebooks/{taskid}',
                      metric=metric)
pcoa_file = f'notebooks/{taskid}/{taskid}.qzv'
pcoa_obj.visualization.save(pcoa_file)

/ClusterApp/api/utils.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  sample_metadata.columns = sample_metadata.columns.str.replace('\s', '_')


'notebooks/a0b4377f6e4540268602c30fedf69b0b/a0b4377f6e4540268602c30fedf69b0b.qzv'

In [17]:
!ls notebooks/example

MSHUB_feature_table_Metadata_combined_blank_subtracted_sample_vs_blank_for_MA.csv


In [19]:
feat_table = pd.read_csv('notebooks/example/MSHUB_feature_table_Metadata_combined_blank_subtracted_sample_vs_blank_for_MA.csv')
feat_table.head()

,filename,ATTRIBUTE_sample,ATTRIBUTE_Source,ATTRIBUTE_GasProduction24hmLg,ATTRIBUTE_GPcat,ATTRIBUTE_CH4ppm,ATTRIBUTE_CH424hmLg,ATTRIBUTE_CH4Ycat,ATTRIBUTE_TVFAcat,ATTRIBUTE_Acecat,...,1041_0.37,1043_0.37,1044_0.37,1045_0.37,1049_0.37,1050_0.37,1051_0.37,1052_0.37,1053_0.37,1054_0.37
0,1.mzML,Sample,Asparagopsis taxiformis,114.7,low,82.0,0.11,low,low,low,...,175.739959,7439.671710,301.370004,2461.011948,11726.78016,23.012886,0.0000,30.300921,0.000000,0.0
1,10.mzML,Sample,Mastocarpus papillatus,130.5,norm,6825.0,10.65,norm,norm,norm,...,0.000000,0.000000,473.284180,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.0
2,11.mzML,Sample,Sargassum fluitans,131.4,norm,6751.0,10.60,norm,norm,norm,...,0.000000,25.133266,3.224939,46.901519,0.00000,0.000000,87787.0909,0.000000,88.535568,0.0
3,12.mzML,Sample,Dictyopteris undulata,129.5,norm,7625.0,11.84,norm,norm,norm,...,17989.395940,1384.704414,1980.594277,0.000000,0.00000,136.514312,0.0000,21.122395,202.982616,0.0
4,13.mzML,Sample,Soleria tenera,126.4,norm,5849.0,13.86,norm,norm,norm,...,0.000000,1115.182126,213.789898,15.078425,0.00000,0.000000,0.0000,0.000000,0.000000,0.0


In [20]:
last_attr = feat_table.columns[feat_table.columns.str.contains('ATTRIBUTE')][-1]
plast_attr = feat_table.columns.get_loc(last_attr)+1

In [21]:
feat_table.columns[plast_attr:]

Index(['4_0.03', '5_0.03', '8_0.04', '9_0.04', '11_0.04', '14_0.04', '15_0.04',
       '16_0.04', '18_0.04', '19_0.04',
       ...
       '1041_0.37', '1043_0.37', '1044_0.37', '1045_0.37', '1049_0.37',
       '1050_0.37', '1051_0.37', '1052_0.37', '1053_0.37', '1054_0.37'],
      dtype='object', length=974)

In [22]:
feat_table.columns[:plast_attr]

Index(['filename', 'ATTRIBUTE_sample', 'ATTRIBUTE_Source',
       'ATTRIBUTE_GasProduction24hmLg', 'ATTRIBUTE_GPcat', 'ATTRIBUTE_CH4ppm',
       'ATTRIBUTE_CH424hmLg', 'ATTRIBUTE_CH4Ycat', 'ATTRIBUTE_TVFAcat',
       'ATTRIBUTE_Acecat', 'ATTRIBUTE_Propcat', 'ATTRIBUTE_Butcat'],
      dtype='object')

In [59]:
meta = feat_table[feat_table.columns[:plast_attr]]
meta.head()

,filename,ATTRIBUTE_sample,ATTRIBUTE_Source,ATTRIBUTE_GasProduction24hmLg,ATTRIBUTE_GPcat,ATTRIBUTE_CH4ppm,ATTRIBUTE_CH424hmLg,ATTRIBUTE_CH4Ycat,ATTRIBUTE_TVFAcat,ATTRIBUTE_Acecat,ATTRIBUTE_Propcat,ATTRIBUTE_Butcat
0,1.mzML,Sample,Asparagopsis taxiformis,114.7,low,82.0,0.11,low,low,low,high,high
1,10.mzML,Sample,Mastocarpus papillatus,130.5,norm,6825.0,10.65,norm,norm,norm,norm,norm
2,11.mzML,Sample,Sargassum fluitans,131.4,norm,6751.0,10.60,norm,norm,norm,norm,norm
3,12.mzML,Sample,Dictyopteris undulata,129.5,norm,7625.0,11.84,norm,norm,norm,norm,norm
4,13.mzML,Sample,Soleria tenera,126.4,norm,5849.0,13.86,norm,norm,norm,norm,norm


In [60]:
feat_table[feat_table.columns[plast_attr:]].head()

,4_0.03,5_0.03,8_0.04,9_0.04,11_0.04,14_0.04,15_0.04,16_0.04,18_0.04,19_0.04,...,1041_0.37,1043_0.37,1044_0.37,1045_0.37,1049_0.37,1050_0.37,1051_0.37,1052_0.37,1053_0.37,1054_0.37
0,0.0,0.000000,0.0,16936.39038,0.0,0.000000,0.000000,1.749039e+03,0.000000,101.425961,...,175.739959,7439.671710,301.370004,2461.011948,11726.78016,23.012886,0.0000,30.300921,0.000000,0.0
1,0.0,474.631334,0.0,15989.45965,0.0,189.385019,467.383513,1.374174e+03,0.000000,500.578299,...,0.000000,0.000000,473.284180,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.0
2,0.0,463.777218,0.0,15702.71154,0.0,1404.873733,474.628584,1.285307e+03,0.000000,483.729759,...,0.000000,25.133266,3.224939,46.901519,0.00000,0.000000,87787.0909,0.000000,88.535568,0.0
3,0.0,422.888693,0.0,15518.84295,0.0,988.605355,258.012614,1.921113e+03,166.363801,0.000000,...,17989.395940,1384.704414,1980.594277,0.000000,0.00000,136.514312,0.0000,21.122395,202.982616,0.0
4,0.0,0.000000,0.0,16127.94969,0.0,193.914708,450.153876,1.103062e+06,0.000000,0.000000,...,0.000000,1115.182126,213.789898,15.078425,0.00000,0.000000,0.0000,0.000000,0.000000,0.0


In [61]:
feat = feat_table[feat_table.columns[plast_attr:]].T
feat.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
4_0.03,0.00000,0.000000,0.000000,0.000000,0.00000,172488.650600,112058.41380,108233.87960,128255.584700,0.00000,...,0.000000,0.00000,19020.070140,26483.691650,0.000000,0.000000,0.00000,7330.645832,7330.645832,0.000000
5_0.03,0.00000,474.631334,463.777218,422.888693,0.00000,2489.716948,0.00000,0.00000,536.444906,0.00000,...,487.912163,0.00000,1586.530971,2206.999134,793.458288,469.568253,467.55038,664.468656,664.468656,445.560603
8_0.04,0.00000,0.000000,0.000000,0.000000,0.00000,916553.944700,0.00000,0.00000,0.000000,0.00000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
9_0.04,16936.39038,15989.459650,15702.711540,15518.842950,16127.94969,15162.867990,21687.25838,18773.31837,13260.339300,18989.32184,...,795656.523700,18639.05164,725957.699600,17993.018130,17669.276500,16017.958720,15886.99707,1478.749531,1478.749531,0.000000
11_0.04,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000


In [63]:
# Have to add because this is used 
# in a filtering step
meta.filename+' Peak area'

0              1.mzML Peak area
1             10.mzML Peak area
2             11.mzML Peak area
3             12.mzML Peak area
4             13.mzML Peak area
                ...            
71             8.mzML Peak area
72             9.mzML Peak area
73         Blank.mzML Peak area
74    Blank_copy.mzML Peak area
75        Blank1.mzML Peak area
Name: filename, Length: 76, dtype: object

In [64]:
feat.columns = meta.filename+' Peak area'
feat.head()

filename,1.mzML Peak area,10.mzML Peak area,11.mzML Peak area,12.mzML Peak area,13.mzML Peak area,14.mzML Peak area,15.mzML Peak area,16.mzML Peak area,17.mzML Peak area,18.mzML Peak area,...,7.mzML Peak area,70.mzML Peak area,71.mzML Peak area,72.mzML Peak area,73.mzML Peak area,8.mzML Peak area,9.mzML Peak area,Blank.mzML Peak area,Blank_copy.mzML Peak area,Blank1.mzML Peak area
4_0.03,0.00000,0.000000,0.000000,0.000000,0.00000,172488.650600,112058.41380,108233.87960,128255.584700,0.00000,...,0.000000,0.00000,19020.070140,26483.691650,0.000000,0.000000,0.00000,7330.645832,7330.645832,0.000000
5_0.03,0.00000,474.631334,463.777218,422.888693,0.00000,2489.716948,0.00000,0.00000,536.444906,0.00000,...,487.912163,0.00000,1586.530971,2206.999134,793.458288,469.568253,467.55038,664.468656,664.468656,445.560603
8_0.04,0.00000,0.000000,0.000000,0.000000,0.00000,916553.944700,0.00000,0.00000,0.000000,0.00000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
9_0.04,16936.39038,15989.459650,15702.711540,15518.842950,16127.94969,15162.867990,21687.25838,18773.31837,13260.339300,18989.32184,...,795656.523700,18639.05164,725957.699600,17993.018130,17669.276500,16017.958720,15886.99707,1478.749531,1478.749531,0.000000
11_0.04,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000


In [65]:
feat.shape

(974, 76)

In [66]:
feat_tmp = pd.DataFrame(feat.index)
feat_tmp.reset_index(inplace=True)
feat_tmp

,index,0
0,0,4_0.03
1,1,5_0.03
2,2,8_0.04
3,3,9_0.04
4,4,11_0.04
...,...,...
969,969,1050_0.37
970,970,1051_0.37
971,971,1052_0.37
972,972,1053_0.37


In [67]:
feat_tmp = pd.DataFrame(feat_tmp[0].apply(lambda a: a.split("_")).tolist())
feat_tmp.reset_index(inplace=True)

feat_tmp.columns = ['row ID', 'row m/z', 'row retention time']
feat_tmp

,row ID,row m/z,row retention time
0,0,4,0.03
1,1,5,0.03
2,2,8,0.04
3,3,9,0.04
4,4,11,0.04
...,...,...,...
969,969,1050,0.37
970,970,1051,0.37
971,971,1052,0.37
972,972,1053,0.37


In [68]:
feat = pd.concat([feat_tmp, feat.reset_index(drop=True)], axis=1)
feat

,row ID,row m/z,row retention time,1.mzML Peak area,10.mzML Peak area,11.mzML Peak area,12.mzML Peak area,13.mzML Peak area,14.mzML Peak area,15.mzML Peak area,...,7.mzML Peak area,70.mzML Peak area,71.mzML Peak area,72.mzML Peak area,73.mzML Peak area,8.mzML Peak area,9.mzML Peak area,Blank.mzML Peak area,Blank_copy.mzML Peak area,Blank1.mzML Peak area
0,0,4,0.03,0.000000,0.000000,0.000000,0.000000,0.00000,172488.650600,112058.413800,...,0.000000,0.00000,19020.070140,26483.691650,0.000000,0.000000,0.000000,7330.645832,7330.645832,0.000000
1,1,5,0.03,0.000000,474.631334,463.777218,422.888693,0.00000,2489.716948,0.000000,...,487.912163,0.00000,1586.530971,2206.999134,793.458288,469.568253,467.550380,664.468656,664.468656,445.560603
2,2,8,0.04,0.000000,0.000000,0.000000,0.000000,0.00000,916553.944700,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,9,0.04,16936.390380,15989.459650,15702.711540,15518.842950,16127.94969,15162.867990,21687.258380,...,795656.523700,18639.05164,725957.699600,17993.018130,17669.276500,16017.958720,15886.997070,1478.749531,1478.749531,0.000000
4,4,11,0.04,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,969,1050,0.37,23.012886,0.000000,0.000000,136.514312,0.00000,0.000000,25209.642230,...,115719.904400,0.00000,0.000000,1206.677247,9784.904506,57232.854400,0.000000,666.802309,666.802309,0.000000
970,970,1051,0.37,0.000000,0.000000,87787.090900,0.000000,0.00000,0.000000,0.000000,...,0.000000,338456.95690,0.000000,0.000000,417207.050100,830.539509,0.000000,170.431159,170.431159,0.000000
971,971,1052,0.37,30.300921,0.000000,0.000000,21.122395,0.00000,0.000000,9372.945875,...,58598.975850,50623.31243,0.000000,1793.250887,0.000000,0.000000,120.965726,185.993803,185.993803,0.000000
972,972,1053,0.37,0.000000,0.000000,88.535568,202.982616,0.00000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,147.198131,147.198131,0.000000


In [69]:
meta.shape

(76, 12)

In [70]:
feat.shape

(974, 79)

In [71]:
metric

'euclidean'

In [72]:
df = feat
df2 = df[df.columns[df.columns.str.contains(' Peak area')]]
df2.columns = [re.sub('(.+\.mzX?ML) .+', '\\1', a) for a in df2.columns]
df2.index = df['row ID'].astype(str)
df2 = df2.T

df2.head()

row ID,0,1,2,3,4,5,6,7,8,9,...,964,965,966,967,968,969,970,971,972,973
1.mzML,0.0,0.000000,0.0,16936.39038,0.0,0.000000,0.000000,1.749039e+03,0.000000,101.425961,...,175.739959,7439.671710,301.370004,2461.011948,11726.78016,23.012886,0.0000,30.300921,0.000000,0.0
10.mzML,0.0,474.631334,0.0,15989.45965,0.0,189.385019,467.383513,1.374174e+03,0.000000,500.578299,...,0.000000,0.000000,473.284180,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.0
11.mzML,0.0,463.777218,0.0,15702.71154,0.0,1404.873733,474.628584,1.285307e+03,0.000000,483.729759,...,0.000000,25.133266,3.224939,46.901519,0.00000,0.000000,87787.0909,0.000000,88.535568,0.0
12.mzML,0.0,422.888693,0.0,15518.84295,0.0,988.605355,258.012614,1.921113e+03,166.363801,0.000000,...,17989.395940,1384.704414,1980.594277,0.000000,0.00000,136.514312,0.0000,21.122395,202.982616,0.0
13.mzML,0.0,0.000000,0.0,16127.94969,0.0,193.914708,450.153876,1.103062e+06,0.000000,0.000000,...,0.000000,1115.182126,213.789898,15.078425,0.00000,0.000000,0.0000,0.000000,0.000000,0.0


In [73]:
pcoa_obj = qiime2PCoA(meta, feat,
                      out_dir=f'notebooks/teste_novo_formato',
                      metric=metric)

/ClusterApp/api/utils.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metadata.rename(index=str, columns={"filename": "#SampleID"},
/ClusterApp/api/utils.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  sample_metadata.columns = sample_metadata.columns.str.replace('\s', '_')
/ClusterApp/api/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metadata.drop(['#SampleID'], axis=1, inplace=True)


In [14]:
os.listdir('notebooks/a0b4377f6e4540268602c30fedf69b0b')

['templates',
 'index.html',
 'img',
 'q2templateassets',
 'js',
 'css',
 'vendor',
 'emperor.html',
 'a0b4377f6e4540268602c30fedf69b0b.qzv']

In [7]:
!ls notebooks/a0b4377f6e4540268602c30fedf69b0b

a0b4377f6e4540268602c30fedf69b0b.qzv


In [8]:
# open the html file on your computer
# copy it if necessary
!firefox a0b4377f6e4540268602c30fedf69b0b/index.html

/bin/sh: 1: firefox: not found
